In [1]:
import sqlite3
import pandas as pd

In [4]:
ori_data = pd.read_csv('./farmersmarkets-2017-01-10.csv')
clean_data = pd.read_csv('./data/address.csv')

In [5]:
conn_ori = sqlite3.connect("farmersmarkets.db") 
ori_data.to_sql('farmersmarkets', conn_ori)
conn_clean = sqlite3.connect("cleanfarmersmarkets.db") 
clean_data.to_sql('cleanfarmersmarkets', conn_clean)

In [6]:
# city, state and county are not empty
empty_address = '''
SELECT FMID, MarketName, city, state, county 
FROM farmersmarkets 
WHERE city IS NULL
OR state IS NULL
OR county IS NULL;
'''
data = pd.read_sql(empty_address, conn_ori)
data

,FMID,MarketName,city,State,County
0,1009364,106 S. Main Street Farmers Market,Six Mile,South Carolina,None
1,1006234,4th Street Farmers Market,Larimer,Colorado,None
2,1006494,52 & Shadeland Avenue Farmers Market,Indianapolis,Indiana,None
3,1009543,ABV Farm Market,Mount Bethe,Pennsylvania,None
4,1001300,Algonquin Farmers Market,Algonquin,Illinois,None
...,...,...,...,...,...
546,1009255,Wood Dale Farmer's Market,Wood Dale,Illinois,None
547,1007624,Worcester Northeast Side Farmers Market,Worcester,Massachusetts,None
548,1009336,Yazoo Farmers Market,Yazoo City,Mississippi,None
549,1009531,Year-Round Cedar City Farmer's Market,Cedar City,Utah,None


In [7]:
empty_address = '''
SELECT FMID, MarketName, city, state, county 
FROM cleanfarmersmarkets 
WHERE city IS NULL
OR state IS NULL
OR county IS NULL;
'''
data = pd.read_sql(empty_address, conn_clean)
data

,FMID,MarketName,City,State,County


In [8]:
same_address = '''
SELECT *
FROM 
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM farmersmarkets
) AS m1,
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM farmersmarkets
) AS m2 
WHERE  m1.zip = m2.zip 
AND m1.county = m2.county 
AND m1.city = m2.city 
AND m1.street = m2.street 
AND m1.state = m2.state 
AND m1.fmid <> m2.fmid; 
'''
data = pd.read_sql(same_address, conn_ori)
data

,FMID,MarketName,zip,street,city,county,state,FMID,MarketName,zip,street,city,county,state
0,1009719,CIty of Momence,60954,123 W. River St.,Momence,Kankakee,Illinois,1001115,Momence Farmer's Market,60954,123 W. River St.,Momence,Kankakee,Illinois
1,1000865,Columbus Winter Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio,1008799,Edible Thursdays Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio
2,1008799,Edible Thursdays Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio,1000865,Columbus Winter Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio
3,1004024,Farmer's Market @ Woburn's Spence Farm,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts,1006893,Woburn Farmers Market,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts
4,1004024,Farmer's Market @ Woburn's Spence Farm,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts,1008931,Woburn Indoor Farmers Market,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts
5,1000396,Four Seasons Market - Richardson,75080,677 W. Campbell Rd,Richardson,Dallas,Texas,1009996,Richardson Farmers Market,75080,677 W. Campbell Rd,Richardson,Dallas,Texas
6,1007285,Hamilton Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama,1008856,Marion County Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama
7,1011624,Main Street Farmers & Artists Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland,1005103,Main Street Farmers Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland
8,1005103,Main Street Farmers Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland,1011624,Main Street Farmers & Artists Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland
9,1008856,Marion County Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama,1007285,Hamilton Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama


In [9]:
same_address = '''
SELECT *
FROM 
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM cleanfarmersmarkets
) AS m1,
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM cleanfarmersmarkets
) AS m2 
WHERE  m1.zip = m2.zip 
AND m1.county = m2.county 
AND m1.city = m2.city 
AND m1.street = m2.street 
AND m1.state = m2.state 
AND m1.fmid <> m2.fmid; 
'''
data = pd.read_sql(same_address, conn_clean)
data

,FMID,MarketName,zip,street,city,county,state,FMID,MarketName,zip,street,city,county,state
